In [1]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [8]:
# 生成数据集，y = Xw + b + 噪声
def synthetic_data(b,w,number_examples):
    #定义X
    X = torch.normal(0,1,size=(number_examples,len(w))) 
    #定义y
    y = torch.mv(X,w)+b
    #引入噪声
    y += torch.normal(0,0.01,size=y.shape)
    return X,y.reshape((-1,1))

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = synthetic_data(true_b,true_w,1000)

In [12]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size): #从0开始，到num_examples结束，步长为batch_size
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [20]:
for X,y in data_iter(10,features,labels):
    print(X,'\n',y)
    break

tensor([[-1.1804, -0.7791],
        [-1.2021,  0.7931],
        [-0.1562,  0.7387],
        [-1.4864, -0.8616],
        [-1.0832, -0.6858],
        [-0.1403,  0.5802],
        [-2.3612, -0.5825],
        [ 0.6724, -0.5949],
        [ 1.5793, -0.4039],
        [-0.6536, -0.8082]]) 
 tensor([[ 4.4843],
        [-0.9027],
        [ 1.3743],
        [ 4.1357],
        [ 4.3758],
        [ 1.9437],
        [ 1.4545],
        [ 7.5617],
        [ 8.7213],
        [ 5.6423]])


In [3]:
# 初始化模型参数
w = torch.normal(0,0.01,size=(2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

In [16]:
batch_size = 10
# 定义模型
def linreg(X,w,b):
    return torch.mm(X,w)+b #使用了广播机制

In [5]:
def squared_loss(y_hat,y):
    return (y_hat - y.reshape(y_hat.shape))**2/2

In [6]:
def sgd(params,lr,batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr* param.grad / batch_size
            param.grad.zero_()

In [21]:
lr = 0.03
num_epochs = 3
net = linreg #定义模型
loss = squared_loss

for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w,b),y)
        l.sum().backward()
        sgd([w,b],lr,batch_size)
    with torch.no_grad(): #接下来的代码，停止梯度计算，因为测试集，不需要进行梯度计算
        train_l = loss(net(features,w,b),labels)
        print(f'epoch {epoch+1},loss {float(train_l.mean()):f}')